In [4]:
!pip install geopandas

     |████████████████████████████████| 994 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 7.5 MB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 32.2 MB/s eta 0:00:01


In [16]:
!brew install spatialindex

==> Downloading https://homebrew.bintray.com/bottles-portable-ruby/portable-ruby-2.6.3_2.yosemite.bottle.tar.gz
######################################################################### 100.0%
==> Pouring portable-ruby-2.6.3_2.yosemite.bottle.tar.gz
==> Tapping homebrew/cask
Cloning into '/usr/local/Homebrew/Library/Taps/homebrew/homebrew-cask'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 554351 (delta 7), reused 4 (delta 2), pack-reused 554335
Receiving objects: 100% (554351/554351), 243.90 MiB | 23.22 MiB/s, done.
Resolving deltas: 100% (391476/391476), done.
Tapped 3855 casks (3,976 files, 521.7MB).
==> Downloading https://homebrew.bintray.com/bottles/spatialindex-1.9.3.catalina
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/fbcbfaf6510137f3168a0
######################################################################## 100.0%
==> Pouring spatialindex-1.9.3.catalina.bo

In [17]:
!pip install rtree

In [19]:
!pip install folium

     |████████████████████████████████| 94 kB 2.0 MB/s eta 0:00:01


In [20]:
#importing libraries
import numpy as np
import pandas as pd
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pandas import DatetimeIndex, Series, to_datetime
import seaborn as sns
import shapely
import geopandas as gpd
import folium
import json
import os
from branca.colormap import linear
import branca.colormap
import rtree
import folium.plugins # The Folium Javascript Map Library
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

## San Francisco Crime Part 2: Geographic Information Systems

### 2) Loading the Data

In [21]:
police = pd.read_csv('../Problem-Sets/data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')
police

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,01/19/2015,14:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,"(37.7617007179518, -122.42158168137)",15006027571000
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821003074
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821004014
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821015200
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,01/27/2015,19:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,"(37.8004687042875, -122.431118543788)",15009822628160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215019,170106724,ASSAULT,BATTERY,Tuesday,02/07/2017,10:08,TENDERLOIN,NONE,TURK ST / HYDE ST,-122.415695,37.782585,"(37.78258519684673, -122.41569492213362)",17010672404134
2215020,170115747,ASSAULT,BATTERY,Friday,02/10/2017,10:28,MISSION,NONE,1600 Block of BRYANT ST,-122.410478,37.766246,"(37.76624632091369, -122.41047806948747)",17011574704134
2215021,170177565,ASSAULT,BATTERY,Wednesday,03/01/2017,00:01,SOUTHERN,NONE,MISSION ST / 4TH ST,-122.404270,37.784479,"(37.784478853874475, -122.40427017948574)",17017756504134
2215022,170183506,VEHICLE THEFT,STOLEN TRUCK,Friday,03/03/2017,20:30,PARK,NONE,1400 Block of GROVE ST,-122.440483,37.775553,"(37.77555290168326, -122.44048310699829)",17018350607025


### 3) Data Exploration and Preparation

#### 3.1 Plot individual incidents in 2018 as points on a map of San Francisco. Be sure to include the following: 

1. Does crime seem randomly distributed in space, or do incidents tend to cluster close together?

2. Share the points by type of crime and analyze whether certain neighborhoods experience certain types of crime more often. 

3. Propose social scientific explanations for the patterns that you find. 



In [22]:
sf_coords = (37.7749, -122.4194) 
sf_map = folium.Map(location=sf_coords, zoom_start=12.4999) 


In [23]:
sf_map

In [24]:
police_sample = police.sample(2000)

In [25]:
police['Category'].unique()

array(['NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'SECONDARY CODES',
       'VANDALISM', 'BURGLARY', 'LARCENY/THEFT', 'DRUG/NARCOTIC',
       'WARRANTS', 'VEHICLE THEFT', 'OTHER OFFENSES', 'WEAPON LAWS',
       'ARSON', 'MISSING PERSON', 'DRIVING UNDER THE INFLUENCE',
       'SUSPICIOUS OCC', 'RECOVERED VEHICLE', 'DRUNKENNESS', 'TRESPASS',
       'FRAUD', 'DISORDERLY CONDUCT', 'SEX OFFENSES, FORCIBLE',
       'FORGERY/COUNTERFEITING', 'KIDNAPPING', 'EMBEZZLEMENT',
       'STOLEN PROPERTY', 'LIQUOR LAWS', 'FAMILY OFFENSES', 'LOITERING',
       'BAD CHECKS', 'TREA', 'GAMBLING', 'RUNAWAY', 'BRIBERY',
       'PROSTITUTION', 'PORNOGRAPHY/OBSCENE MAT',
       'SEX OFFENSES, NON FORCIBLE', 'SUICIDE', 'EXTORTION'], dtype=object)

In [26]:
police_sample.groupby('Category').count().sort_values(by=['IncidntNum'], ascending=False).head(5)

,IncidntNum,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
Category,,,,,,,,,,,,
LARCENY/THEFT,401,401,401,401,401,401,401,401,401,401,401,401
OTHER OFFENSES,300,300,300,300,300,300,300,300,300,300,300,300
NON-CRIMINAL,213,213,213,213,213,213,213,213,213,213,213,213
ASSAULT,193,193,193,193,193,193,193,193,193,193,193,193
DRUG/NARCOTIC,114,114,114,114,114,114,114,114,114,114,114,114


In [27]:
def color(category):
    if category == "LARCENY/THEFT":
        return 'green'
    if category == 'NON-CRIMINAL':
        return 'blue'
    if category == "ASSAULT":
        return  'cyan'
    if category == 'VEHICLE THEFT':
        return 'black'
    elif category == 'OTHER OFFENSES': 
        return 'yellow'
    else:
        return 'magenta'

In [28]:
for index, row in police_sample.iterrows():
     folium.CircleMarker([row['Y'], row['X']],
                          radius=1, fill = True,
                          fill_color=color(row['Category']), color = color(row['Category']), fill_opacity=0.7).add_to(sf_map)

In [29]:
sf_map

1. Does crime seem randomly distributed in space, or do incidents tend to cluster close together?

Based on our plot above, we see that crimes tend to cluster together. In particular, the magenta
colored points are mostly concentrated in the Downtown/Civic Center neighborhood of SF. Then as we move further away from this neighborhood the points seem to spread out and are not as clustered. This must indicate 
that certain areas are more suceptible to crime due to neighborhood makeup/density and/or socieconomic reasons. 


2. Share the points by type of crime and analyze whether certain neighborhoods experience certain types of crime more often. 

In our map, green indicates larceny/theft, blue indicates non-criminal, light blue (cyan) indicates assault, 
black indicates other offenses, and magenta indicates any other crime not listed above. One thing that I noticed in the map is that there are more larceny/theft incidents that occur in the Dowtown/Civic center neighborhood than other neighborhoods in SF. For instance, the Outer Sunset/Parkside neighborhoods don't have many larceny/theft incidents. A reason this might be the case is due to a higher concentration of people in the Downtown/Civic Center area than in a residential area like Outer Sunset/Parkside. There are more people and more tourists where it would make it easier to commit a crime like larceny/theft. 

3. Propose social scientific explanations for the patterns that you find. 

More crime can be indicative of higher concentrations of people, in areas like Downtown, causing 
police to heavily patrol these areas. Not to mention, the socioeconomic makeup of neighborhood can also 
help determine why there is more crime in one are than another. Particularly, neighborhoods like Outer Sunset/Parkside and Presidio have less concentration of crime, but have a higher concentration of white residents. On the other hand, neighborhoods like Downtown and Mission have a higher concentration of crime and have higher concentration of Latinx/Black residents. 

#### 3.2 Merge the incidents data with either a Shapefile or GeoJSON file with information on the boundaries of neighborhoods in San Francisco 

In [30]:
#uses shapely - to manipulate planar 
neighborhood_data = os.path.join('data/SF-Find-Neighborhoods.geojson')
geo_json_data = json.load(open(neighborhood_data))
neighborhood_shapes = gpd.read_file('data/SF-Find-Neighborhoods.geojson')
neighborhood_shapes.head()

,link,name,geometry
0,"http://en.wikipedia.org/wiki/Sea_Cliff,_San_Fr...",Seacliff,"MULTIPOLYGON (((-122.49346 37.78352, -122.4937..."
1,None,Lake Street,"MULTIPOLYGON (((-122.48715 37.78379, -122.4872..."
2,http://www.nps.gov/prsf/index.htm,Presidio National Park,"MULTIPOLYGON (((-122.47758 37.81099, -122.4771..."
3,None,Presidio Terrace,"MULTIPOLYGON (((-122.47241 37.78735, -122.4710..."
4,http://www.sfgate.com/neighborhoods/sf/innerri...,Inner Richmond,"MULTIPOLYGON (((-122.47263 37.78631, -122.4668..."


In [31]:
police_geo = police_sample.apply(lambda x : shapely.geometry.Point([x['X'],x['Y']]), axis=1)
police_geodf = gpd.GeoDataFrame(police_sample, geometry = police_geo)
police_geodf.crs = {'init' :'epsg:4326'}
police_geodf.head()

/Users/jarellymartin/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,geometry
1892553,180219709,ASSAULT,THREATS AGAINST LIFE,Saturday,03/24/2018,10:20,MISSION,"ARREST, BOOKED",4000 Block of 18TH ST,-122.434457,37.760977,"(37.760976609084544, -122.43445735395501)",18021970919057,POINT (-122.43446 37.76098)
501609,60965808,VANDALISM,"MALICIOUS MISCHIEF, BREAKING WINDOWS",Friday,09/08/2006,17:00,MISSION,NONE,2600 Block of 25TH ST,-122.404729,37.751761,"(37.7517606134232, -122.404728798761)",6096580828100,POINT (-122.40473 37.75176)
384201,136235901,NON-CRIMINAL,LOST PROPERTY,Friday,12/13/2013,11:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",13623590171000,POINT (-122.40340 37.77542)
715881,30688381,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Sunday,06/01/2003,11:00,INGLESIDE,NONE,100 Block of RUSSIA AV,-122.435941,37.721062,"(37.7210616750407, -122.435941304691)",3068838106242,POINT (-122.43594 37.72106)
1845899,170562433,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,07/11/2017,12:24,SOUTHERN,NONE,0 Block of 12TH ST,-122.420001,37.774095,"(37.774094856688166, -122.42000143696421)",17056243364020,POINT (-122.42000 37.77409)


In [32]:

neighborhood_and_police = gpd.sjoin(police_geodf, neighborhood_shapes, op='within', how="inner")


<ipython-input-32-50e45acbed77>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  neighborhood_and_police = gpd.sjoin(police_geodf, neighborhood_shapes, op='within', how="inner")


In [33]:
neighborhood_and_police

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,geometry,index_right,link,name
1892553,180219709,ASSAULT,THREATS AGAINST LIFE,Saturday,03/24/2018,10:20,MISSION,"ARREST, BOOKED",4000 Block of 18TH ST,-122.434457,37.760977,"(37.760976609084544, -122.43445735395501)",18021970919057,POINT (-122.43446 37.76098),37,http://www.sfgate.com/neighborhoods/sf/castro/,Castro
2055203,170904738,"SEX OFFENSES, FORCIBLE",SEXUAL BATTERY,Tuesday,10/31/2017,16:00,MISSION,NONE,CASTRO ST / 18TH ST,-122.435003,37.760888,"(37.76088780611937, -122.43500286429793)",17090473804144,POINT (-122.43500 37.76089),37,http://www.sfgate.com/neighborhoods/sf/castro/,Castro
1367115,110719199,BURGLARY,"BURGLARY OF APARTMENT HOUSE, UNLAWFUL ENTRY",Wednesday,09/07/2011,13:55,MISSION,"ARREST, BOOKED",100 Block of HARTFORD ST,-122.433754,37.760145,"(37.7601445174796, -122.433753565044)",11071919905013,POINT (-122.43375 37.76014),37,http://www.sfgate.com/neighborhoods/sf/castro/,Castro
1328098,60179112,BURGLARY,"BURGLARY,STORE UNDER CONSTRUCTION, UNLAWFUL ENTRY",Sunday,02/12/2006,11:00,MISSION,NONE,4200 Block of 18TH ST,-122.436657,37.760736,"(37.7607356573712, -122.436657375769)",6017911205153,POINT (-122.43666 37.76074),37,http://www.sfgate.com/neighborhoods/sf/castro/,Castro
360692,120421362,OTHER OFFENSES,RESISTING ARREST,Monday,05/28/2012,01:55,MISSION,NONE,400 Block of CASTRO ST,-122.435150,37.761760,"(37.7617597243598, -122.435150099812)",12042136227170,POINT (-122.43515 37.76176),37,http://www.sfgate.com/neighborhoods/sf/castro/,Castro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280980,96128090,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Tuesday,12/08/2009,15:00,PARK,NONE,400 Block of CORBETT AV,-122.445573,37.757909,"(37.7579092860723, -122.445573459996)",9612809006244,POINT (-122.44557 37.75791),46,http://en.wikipedia.org/wiki/Neighborhoods_in_...,Clarendon Heights
1923902,166020948,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,01/24/2016,13:00,PARK,NONE,TWINPEAKS BL / BURNETT AV,-122.446951,37.756549,"(37.75654865930111, -122.44695083457121)",16602094806244,POINT (-122.44695 37.75655),46,http://en.wikipedia.org/wiki/Neighborhoods_in_...,Clarendon Heights
940306,130431555,NON-CRIMINAL,FOUND PROPERTY,Saturday,05/25/2013,17:44,TARAVAL,NONE,0 Block of SAINT FRANCIS BL,-122.471013,37.734833,"(37.7348332492305, -122.471013306253)",13043155572000,POINT (-122.47101 37.73483),115,"http://en.wikipedia.org/wiki/St._Francis_Wood,...",St. Francis Wood
1929955,180316480,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Sunday,04/29/2018,05:45,PARK,NONE,300 Block of OAKPARK DR,-122.458856,37.754013,"(37.75401286602956, -122.45885596524992)",18031648028160,POINT (-122.45886 37.75401),110,http://en.wikipedia.org/wiki/Neighborhoods_in_...,Forest Knolls


### 4) Mapping

#### 4.1 Construct a choropleth map, coloring in each neighborhood by how mnay incidents it had in 2017. Then, construct several maps that explore differences by day of week, time of years, time of day, etc.

In [34]:
neighborhood_and_police['Date'] = pd.to_datetime(neighborhood_and_police['Date'])

In [35]:
only_2017 = neighborhood_and_police[neighborhood_and_police['Date'].dt.year ==2017]

In [36]:
sf = folium.Map(location=(37.7749, -122.4194) , zoom_start=12.4999) 

folium.Choropleth(
    geo_data=geo_json_data,
    data=only_2017,
    columns=['name','IncidntNum'],
    key_on='feature.properties.name',
    fill_color='YlGn'
).add_to(sf)

sf

Here, I am looking at Friday and the number of incidents per neighborhood. 

In [37]:
only_friday = neighborhood_and_police[neighborhood_and_police['DayOfWeek']=='Friday']

In [38]:
sf_friday = folium.Map(location=(37.7749, -122.4194) , zoom_start=12.4999) 

folium.Choropleth(
    geo_data=geo_json_data,
    data=only_friday,
    columns=['name','IncidntNum'],
    key_on='feature.properties.name',
    fill_color='YlGn'
).add_to(sf_friday)

sf_friday

Here, I am looking at Wednesday and the number of incidents per neighborhood. 

In [39]:
only_wed = neighborhood_and_police[neighborhood_and_police['DayOfWeek']=='Wednesday']
sf_wed = folium.Map(location=(37.7749, -122.4194) , zoom_start=12.4999) 

folium.Choropleth(
    geo_data=geo_json_data,
    data=only_wed,
    columns=['name','IncidntNum'],
    key_on='feature.properties.name',
    fill_color='YlGn'
).add_to(sf_wed)

sf_wed

Here, I am looking at only summer. 

In [40]:
 summer = neighborhood_and_police[( neighborhood_and_police['Date'].dt.month > 6) & ( neighborhood_and_police['Date'].dt.month < 10)]

In [41]:
sf_summer = folium.Map(location=(37.7749, -122.4194) , zoom_start=12.4999) 

folium.Choropleth(
    geo_data=geo_json_data,
    data=summer,
    columns=['name','IncidntNum'],
    key_on='feature.properties.name',
    fill_color='YlGn'
).add_to(sf_summer)

sf_summer

Here, I am looking at only Fall. 

In [42]:
fall = neighborhood_and_police[( neighborhood_and_police['Date'].dt.month > 7) & ( neighborhood_and_police['Date'].dt.month <= 12)]

In [43]:
sf_fall = folium.Map(location=(37.7749, -122.4194) , zoom_start=12.4999) 

folium.Choropleth(
    geo_data=geo_json_data,
    data=fall,
    columns=['name','IncidntNum'],
    key_on='feature.properties.name',
    fill_color='YlGn'
).add_to(sf_fall)

sf_fall

#### 4.2 Do you notice any patterns? Are there particular neighborhoods where crime concentrates more heavily?

In the sf_friday map there are particular neighborhoods where there is a higher number of incidents than in the sf_wednesday map. For instance, when looking at the outer/inner richmond neighborhoods those neighborhoods are a darker green which means there are more incidents. On the other hand, when looking at the wednesday map for those particular neighborhoods they are a yellow/light green color inidicating less indicendts. Also, when looking at twin peaks/diamond heights/noe valley neighborhoods on the friday map there are more indicents while on the wednesday map those areas are shaded black indicating no incidents at all. 

During the summer, we see in the map that there's a number of incidents that occur in the South of Market/Potrero Hill than in the map for Fall incidents. In both maps, the Mission/Potrero Hill neighborhoos tend to have the same number of incidents regardless of the time of year. But in general, there are less yellow/light green neighborhoods in the Summer than in the Fall. My hypothesis of why this might be the case is that in the summer there are more people who are outside and on vacation increasing the possibility of a crime/incident.

#### 4.3 Construct a heat map of crime

In [44]:
lat = police_sample['Y'].values 
lon = police_sample['X'].values 
police_coord = np.vstack((lat, lon)).transpose().tolist() 


In [45]:
sf_heat = folium.Map(location=(37.7749, -122.4194) , zoom_start=12.4999) 
sf_heatmap = HeatMap(police_coord, radius = 10) 
sf_heat.add_child(sf_heatmap)

How does the heat map compare to the choropleth map? Are neighborhoods a reasonably good proxy for the actual concentration of crime?

When working with a heat map you are able to see what areas have a higher number of incidents by color. But when you zoom all the way out everythng seems to be a green color, but as you zoom in you are able to tell the difference in the concentration of incidents. Another thing I noticed with heat maps is that as you zoom the incidents just become points where don't get an understanding of the number of incidens in relation to the neighborhood. But when looking at a choropleth map, we get to see the number of incidents in relation to where it occured. This is to say that a very small concentration of incidents in a particular street corner can have a major effect in the overall number of incidents in a neighborhood. In other words, neighborhoods might not be a good proxy if the concentration of crim is only in small street corner like explained above.

### 5) Discussion Questions


#### 5.1 Based on the evidence from the lab assignment why do you think "hot spots" policing became more popular in the last decades? What there pros and cons of this kind of approach?

Based on the heatmap from above, we see that there are concentrations of crime in relatively small areas and street corners instead of a larger area/neighborhood. Since there's more crime in one particular area then it might seem reasonable to only police that one area or "hot spot." But, this might raise some concerns where "hot spots" are being overly policed causing even more crime to occur. Then this just becomes a cycle of crime and not reducing the number of incidents in that area. A pro of this kind of approach is that the whole neighborhood is not being supervised by the police and creating this cycle of crime.

#### 5.4 Comment on what sorts of incidents get reported in this database: 

For instance, do you see a lot of reports about things like white collar crime? How do you think incident categories are selected? As data scientists, what kinds of ethical and legal concerns should we be aware of when we construct these sorts of datasets?

From looking at the different categories, I noticed that a lot of the incidents that are reported tend to fall under "non-criminal." I did a quick Google search about "non-criminal" incidents and learned that traffic violations tend to fall in this category. Then, I did another quick search to learn different types of white collar crimes which are typically: wage theft, bribery, fraud, cybercrime, money laudering, identity theft, and etc. Many of these crimes are actually not represented in the category column, which means that they are not interested in targeting white offenders. Since these categories don't fall under white collar crime, this is suggestive that these categories might be selected due to race. Simply put, these categories are picked by which crimes Black/Latinx offenders commit more often. They disportionately seek black and brown offenders, when the reality is that crime rates are equal regardless of race. When constructing these data sets, data scientists need to concious of the systems that target oppressed people such as Black and Brown offenders.

#### 5.3 What other sorts of data would help improve your analysis?

One thing that would have helped with my analysis of crime is having access to information about a particular neighborhood. I had to go to Google in order to get an understanding of the neighborhood's demographics to build upon my analysis of crime and its relation to socieconomic status. I could have created a choropleth map where I took into account the racial makeup of a neighborhood and compared it to the choropleth map of just crime. 